In [ ]:
import os
from dotenv import load_dotenv
import bs4
from langchain_community.llms import Ollama
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It',groq_api_key= groq_api_key)

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage
model.invoke([HumanMessage(content="Hi I am Aniket and I am a data scientist")])

AIMessage(content="Hello Aniket, \n\nIt's nice to meet you! That's great, data science is a fascinating field. \n\nWhat kind of data science work do you do? Are you working on any interesting projects right now? \n\nI'm eager to learn more about your work.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 21, 'total_tokens': 86, 'completion_time': 0.118181818, 'prompt_time': 0.00019284, 'queue_time': 0.00382567, 'total_time': 0.118374658}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-abc66191-a2e5-446f-8367-769c77c97eb5-0', usage_metadata={'input_tokens': 21, 'output_tokens': 65, 'total_tokens': 86})

In [17]:
model.invoke(
    [
        HumanMessage(content="hI I AM A DATA SCIENTIST"),
        AIMessage(content="Hi Aniket, it's nice to meet you!\n\nThat's great! Data science is a fascinating field. What kind of work do you do as a data scientist? \n\nAre you working on any interesting projects right now? I'd love to hear more about your work.\n"),
        HumanMessage(content="hey what is my name and what do I do?")

    ]
)

AIMessage(content="You told me your name is Aniket, and that you are a data scientist!  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more. 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 100, 'total_tokens': 150, 'completion_time': 0.090909091, 'prompt_time': 0.002572932, 'queue_time': 0.004007168, 'total_time': 0.093482023}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-90bedbeb-1d48-456c-b258-7dbae8e5f14c-0', usage_metadata={'input_tokens': 100, 'output_tokens': 50, 'total_tokens': 150})

In [36]:
# Add message history to make it stateful
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [37]:
config ={"session_id":"chat1"}

In [40]:
with_message_history.invoke(input=
    [HumanMessage(content="Hi I am Aniket an I am data scientist")],
    config=config
)

AIMessage(content="Hi Aniket, it's nice to meet you!\n\nThat's great you're a data scientist. What kind of work do you do? \n\nI'm always interested in learning more about how people use data to solve problems and make insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 20, 'total_tokens': 78, 'completion_time': 0.105454545, 'prompt_time': 9.656e-05, 'queue_time': 0.003416929, 'total_time': 0.105551105}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-63cd4338-34d9-4b32-af68-5b3885e4db6c-0', usage_metadata={'input_tokens': 20, 'output_tokens': 58, 'total_tokens': 78})

with prompt template

In [47]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are an assistant. Answer all the questions in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [48]:
chain.invoke({'messages':[HumanMessage(content="Hi I am Aniket")],
              'language':'Hindi'
              })


AIMessage(content='Namaste Aniket!  😊 \n\nआपके सवालों के जवाब देने के लिए मैं तैयार हूँ।  आप क्या जानना चाहते हैं? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 26, 'total_tokens': 66, 'completion_time': 0.072727273, 'prompt_time': 0.000146069, 'queue_time': 0.00269887, 'total_time': 0.072873342}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9bb004bc-1923-490d-9a70-e89a83dcb3f6-0', usage_metadata={'input_tokens': 26, 'output_tokens': 40, 'total_tokens': 66})

In [49]:
with_message_history = RunnableWithMessageHistory(chain,
                                                  get_session_history,
                                                  input_messages_key='messages')

In [50]:
config = {"configurable":{'session_id':"chat4"}}
response = with_message_history.invoke(
{'messages': HumanMessage(content="Hi I am Aniket"),'language':'Hindi'},
config = config
)
response.content

'नमस्ते अनिकेत! \n\nमुझे आपकी मदद करने में खुशी हो रही है। आप कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि सही उत्तर दे सकूँ। 😊 \n\n'

Manage conversation history

In [6]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy='last',
    token_counter=model,
    include_system = True,
    allow_partials=False,
    start_on = "human"
)